In [1]:
import pandas as pd
import numpy as np
from math import pi
import warnings
warnings.filterwarnings("ignore")
import features

# Weather Data Compress

In [125]:
ws_10 = pd.read_parquet("data/ITU_DATA/prognosis/ENetNEA/wind_speed_10m.parquet")
ws_100 = pd.read_parquet("data/ITU_DATA/prognosis/ENetNEA/wind_speed_100m.parquet")
wd_10 = pd.read_parquet("data/ITU_DATA/prognosis/ENetNEA/wind_direction_10m.parquet")
wd_100 = pd.read_parquet("data/ITU_DATA/prognosis/ENetNEA/wind_direction_100m.parquet")
tmp_2 = pd.read_parquet("data/ITU_DATA/prognosis/ENetNEA/temperatur_2m.parquet")
tmp_100 = pd.read_parquet("data/ITU_DATA/prognosis/ENetNEA/temperatur_100m.parquet")

In [56]:
def compress(df):
    df = df.reset_index().rename({"index":"TIME_CET"}, axis=1)
    df = df.groupby("TIME_CET", as_index=False).last()
    return df

In [58]:
ws_10 = compress(ws_10)
ws_100 = compress(ws_100)
wd_10 = compress(wd_10)
wd_100 = compress(wd_100)
tmp_2 = compress(tmp_2)
tmp_100 = compress(tmp_100)

In [59]:
ws_10.to_parquet("data/ITU_DATA/prognosis/ENetNEA/ws_10m_cmp.parquet")
ws_100.to_parquet("data/ITU_DATA/prognosis/ENetNEA/ws_100m_cmp.parquet")
wd_10.to_parquet("data/ITU_DATA/prognosis/ENetNEA/wd_10m_cmp.parquet")
wd_100.to_parquet("data/ITU_DATA/prognosis/ENetNEA/wd_100m_cmp.parquet")
tmp_2.to_parquet("data/ITU_DATA/prognosis/ENetNEA/tmp_2m_cmp.parquet")
tmp_100.to_parquet("data/ITU_DATA/prognosis/ENetNEA/tmp_100m_cmp.parquet")

# Wind Speed Vector Extract

In [126]:
ws_10 = pd.read_parquet("data/ITU_DATA/prognosis/ENetNEA/ws_10m_cmp.parquet")
ws_100 = pd.read_parquet("data/ITU_DATA/prognosis/ENetNEA/ws_100m_cmp.parquet")
wd_10 = pd.read_parquet("data/ITU_DATA/prognosis/ENetNEA/wd_10m_cmp.parquet")
wd_100 = pd.read_parquet("data/ITU_DATA/prognosis/ENetNEA/wd_100m_cmp.parquet")

In [128]:
def get_uv(ws, wd):
    cols = ws.columns

    df_u = pd.DataFrame(columns=cols)
    df_u['TIME_CET'] = ws["TIME_CET"].to_list()
    df_u['predicted_ahead'] = ws["predicted_ahead"].to_list()
    
    df_v = pd.DataFrame(columns=cols)
    df_v['TIME_CET'] = ws["TIME_CET"].to_list()
    df_v['predicted_ahead'] = ws["predicted_ahead"].to_list()

    grid = ws.columns.drop(["predicted_ahead","TIME_CET"]).astype(str)

    for g in grid: 
        u, v = features.to_vector(ws[g], wd[g])
        df_u[g] = u.to_list() 
        df_v[g] = v.to_list()
    
    return df_u, df_v

In [62]:
df_u, df_v = get_uv(ws_10, wd_10)

In [63]:
df_u.to_parquet("data/ITU_DATA/prognosis/ENetNEA/wu_10m_cmp.parquet")
df_v.to_parquet("data/ITU_DATA/prognosis/ENetNEA/wv_10m_cmp.parquet")

In [64]:
df_u, df_v = get_uv(ws_10, wd_10)

In [65]:
df_u.to_parquet("data/ITU_DATA/prognosis/ENetNEA/wu_100m_cmp.parquet")
df_v.to_parquet("data/ITU_DATA/prognosis/ENetNEA/wv_100m_cmp.parquet")